In [1]:
import os
import pandas as pd
from ultralytics import YOLO, settings

# --- Cấu hình đường dẫn ---
WINDOWS_DRIVE_DATA = "/mnt/f/COCO2017_Data"
os.makedirs(WINDOWS_DRIVE_DATA, exist_ok=True)
settings.update({'datasets_dir': WINDOWS_DRIVE_DATA})

MODEL_CONFIG = "/home/diy-hus/PCBNet_NEW/pcbnet-v1.yaml"
DATA_YAML = "coco.yaml"
PROJECT = "./runs_pcbnet_v1_coco"
NAME = "pretrain_pcbnet_v1_scratch"

# --- Load Model ---
try:
    model = YOLO(MODEL_CONFIG)
except Exception as e:
    print(f"Error loading config: {e}")
    exit()

# --- SỬA LỖI: Hàm Callback xử lý Loss dạng Tensor ---
def print_all_metrics(trainer):
    current_epoch = trainer.epoch + 1
    print(f"\n>>> Full Metrics Report for Epoch {current_epoch} <<<")
    
    # In các metrics cơ bản (mAP, Precision, Recall, etc.)
    if hasattr(trainer, 'metrics'):
        for key, value in trainer.metrics.items():
            print(f"{key:<25}: {value:.5f}")
    
    # In Training Loss (Box, Cls, DFL)
    if hasattr(trainer, 'tloss') and trainer.tloss is not None:
        # Kiểm tra xem tloss là một số (scalar) hay một mảng (tensor nhiều phần tử)
        if trainer.tloss.numel() == 1:
            print(f"{'Training Loss':<25}: {trainer.tloss.item():.5f}")
        else:
            # Chuyển tensor thành list để in ra 3 thành phần loss riêng biệt
            loss_values = trainer.tloss.tolist()
            loss_str = ", ".join([f"{x:.5f}" for x in loss_values])
            print(f"{'Training Losses':<25}: [{loss_str}] (Box, Cls, DFL)")
            
    print("-" * 50)

# Gắn callback vào model
model.add_callback("on_fit_epoch_end", print_all_metrics)

print("Starting training on WSL...")

# --- Bắt đầu Training ---
try:
    results = model.train(
        data=DATA_YAML,
        epochs=100,
        batch=16,
        imgsz=640,
        workers=0,
        amp=False,
        patience=10,
        optimizer='AdamW',
        lr0=0.001,
        cos_lr=True,
        device=0,
        project=PROJECT,
        name=NAME,
        exist_ok=True,
        plots=True,
        save=True,
        save_period=-1
    )
except Exception as e:
    print(f"\nTRAINING FAILED: {e}")
    print("Suggestion: Try reducing 'batch' size if facing OOM errors.")

# --- Hiển thị kết quả sau khi train xong ---
csv_path = os.path.join(PROJECT, NAME, "results.csv")
if os.path.exists(csv_path):
    print(f"\nMetrics saved to CSV: {csv_path}")
    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip()
        print("\nFinal Epoch Metrics:")
        print(df.tail(1).T)
    except Exception as e:
        print(f"Could not read CSV file: {e}")

WARNING ⚠️ no model scale passed. Assuming scale='n'.
Starting training on WSL...
New https://pypi.org/project/ultralytics/8.3.241 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.240 🚀 Python-3.11.14 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=coco.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/home/diy-hus/PCBNet_NEW/